In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/d/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [ ]:
/kaggle/input/d/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv

In [3]:
import os
import json
import pandas as pd
from sklearn.model_selection import train_test_split 


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
data = pd.read_csv("/kaggle/input/d/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

In [7]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [8]:
data.sentiment.value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [9]:
map_value = {'positive':1,'negative':0}

In [12]:
data['sentiment']=data.sentiment.map(map_value)

In [13]:
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [14]:
train_data,test_data = train_test_split(data, test_size=0.35,random_state=42
                                       )

In [15]:
print(train_data.shape,'\n',test_data.shape)

(32500, 2) 
 (17500, 2)


Tokenize the Text Data

In [17]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])

In [18]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']),maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']),maxlen=200)

In [19]:
print(X_train)

[[   0    0    0 ...    6   62    6]
 [   0    0    0 ...   51  592  615]
 [   0    0    0 ...  142  103   11]
 ...
 [   0    0    0 ... 1664    2  608]
 [   0    0    0 ...  246  103  125]
 [   0    0    0 ...   69   71 2093]]


In [20]:
print(X_test)

[[   0    0    0 ...  979  722  155]
 [  12  161   58 ...  376    7    7]
 [   0    0    0 ...   51 1083   98]
 ...
 [1566    5  185 ...   41    4  155]
 [   0    0    0 ...   11   71  912]
 [   0    0    0 ...   11   32  722]]


In [22]:
y_train = train_data['sentiment']
y_test = test_data['sentiment']

Build the Model : LSTM (Long Short Term Memory)

In [23]:
model = Sequential()
model.add(Embedding(input_dim =5000,output_dim=128,input_length =200))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [24]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [26]:
model.compile(optimizer='adam',loss= 'binary_crossentropy',metrics=['accuracy'])

In [28]:
model.fit(X_train,y_train,epochs =10, batch_size=64,validation_split=0.2)

Epoch 1/10
407/407 ━━━━━━━━━━━━━━━━━━━━ 113s 277ms/step - accuracy: 0.7105 - loss: 0.5412 - val_accuracy: 0.8448 - val_loss: 0.3670
Epoch 2/10
407/407 ━━━━━━━━━━━━━━━━━━━━ 112s 275ms/step - accuracy: 0.8586 - loss: 0.3498 - val_accuracy: 0.7877 - val_loss: 0.4569
Epoch 3/10
407/407 ━━━━━━━━━━━━━━━━━━━━ 112s 275ms/step - accuracy: 0.8503 - loss: 0.3546 - val_accuracy: 0.8192 - val_loss: 0.4387
Epoch 4/10
407/407 ━━━━━━━━━━━━━━━━━━━━ 112s 275ms/step - accuracy: 0.8616 - loss: 0.3277 - val_accuracy: 0.8551 - val_loss: 0.3270
Epoch 5/10
407/407 ━━━━━━━━━━━━━━━━━━━━ 142s 275ms/step - accuracy: 0.8977 - loss: 0.2538 - val_accuracy: 0.8745 - val_loss: 0.3228
Epoch 6/10
407/407 ━━━━━━━━━━━━━━━━━━━━ 112s 275ms/step - accuracy: 0.9192 - loss: 0.2128 - val_accuracy: 0.8774 - val_loss: 0.3116
Epoch 7/10
407/407 ━━━━━━━━━━━━━━━━━━━━ 112s 275ms/step - accuracy: 0.9340 - loss: 0.1747 - val_accuracy: 0.8789 - val_loss: 0.3190
Epoch 8/10
407/407 ━━━━━━━━━━━━━━━━━━━━ 112s 276ms/step - accuracy: 0.9462 -

In [29]:
loss,accuracy = model.evaluate(X_test,y_test)
print(f"Test Loss : {loss}")
print(f"Test Accuracy :{accuracy}")

547/547 ━━━━━━━━━━━━━━━━━━━━ 39s 71ms/step - accuracy: 0.8715 - loss: 0.3745
Test Loss : 0.3823173940181732
Test Accuracy :0.8744000196456909


Building A Predictive System


In [39]:
def predict_sentiment(review):
    #tokenize and Pad Review
    sequences = tokenizer.texts_to_sequences(review)
    padded_sequence = pad_sequences(sequences,maxlen=200)
    prediction = model.predict(padded_sequence)
    sentiment = "positive" if prediction[0][0]>0.5 else 'negative'
    return sentiment

In [40]:
data['review'][100]

"This short film that inspired the soon-to-be full length feature - Spatula Madness - is a hilarious piece that contends against similar cartoons yielding multiple writers. The short film stars Edward the Spatula who after being fired from his job, joins in the fight against the evil spoons. This premise allows for some funny content near the beginning, but is barely present for the remainder of the feature. This film's 15-minute running time is absorbed by some odd-ball comedy and a small musical number. Unfortunately not much else lies below it. The plot that is set up doesn't really have time to show. But it's surely follows it plot better than many high-budget Hollywood films. This film is worth watching at least a few times. Take it for what it is, and don't expect a deep story."

In [45]:
new_review2 = data['review'][100]


In [46]:
new_review2

"This short film that inspired the soon-to-be full length feature - Spatula Madness - is a hilarious piece that contends against similar cartoons yielding multiple writers. The short film stars Edward the Spatula who after being fired from his job, joins in the fight against the evil spoons. This premise allows for some funny content near the beginning, but is barely present for the remainder of the feature. This film's 15-minute running time is absorbed by some odd-ball comedy and a small musical number. Unfortunately not much else lies below it. The plot that is set up doesn't really have time to show. But it's surely follows it plot better than many high-budget Hollywood films. This film is worth watching at least a few times. Take it for what it is, and don't expect a deep story."

In [44]:
new_review = "This Movie was Fantastic .I loved it"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is : {sentiment}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 294ms/step
The sentiment of the review is : negative


In [47]:
sentiment = predict_sentiment(new_review2)
print(f"The sentiment of the review is : {sentiment}")

25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step
The sentiment of the review is : negative
